In [1]:
import torchvision
import torchvision.transforms as transforms
import torch.backends.cudnn as cudnn
import torch.optim as optim
import os
import argparse

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
from torch.autograd import Variable
from torchsummary import summary

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [4]:
transform_train = transforms.Compose([
                                      transforms.RandomCrop(32, padding=4),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

transform_test = transforms.Compose([
                                     transforms.ToTensor(),
                                     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

trainset = torchvision.datasets.CIFAR10(
    root='./data',train=True,download=True,transform=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128,shuffle=True,num_workers=2
)
testset = torchvision.datasets.CIFAR10(
    root='./data',train=False,download=True,transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100,shuffle=False,num_workers=2
)
classes=('plane','car','bird','cat','deer','dog','frog','horse','ship','truck')

Files already downloaded and verified
Files already downloaded and verified


In [5]:
class MobileNet(nn.Module):
    def __init__(self, ch_in=3, n_classes=10):
        super(MobileNet,self).__init__()

        def conv_bn(inp, oup, stride):
            return nn.Sequential(
              nn.Conv2d(inp, oup, 3, stride, 1,bias=False),
              nn.BatchNorm2d(oup),
              nn.ReLU(inplace=True)
            )
        # depthwise
        def conv_dw(inp, oup, stride):
            # depthwise convolution
            # group1 : 입출력 모두 convolution
            # group2 : 2그룹으로 나누어 convolution $ concatenation
            return nn.Sequential(
              nn.Conv2d(inp, inp, 3, stride, 1, groups=inp, bias=False),
              nn.BatchNorm2d(inp),
              nn.ReLU(inplace=True),

              # pointwise
              nn.Conv2d(inp, oup, 1, 1, 0, bias=False),
              nn.BatchNorm2d(oup),
              nn.ReLU(inplace=True)
            )

        self.model = nn.Sequential(
            conv_bn(ch_in, 32, 2),
            conv_dw(32, 64, 1),
            conv_dw(64, 128, 2),
            conv_dw(128, 128, 1),
            conv_dw(128, 256, 2),
            conv_dw(256, 256, 1),
            conv_dw(256, 512, 2),
            conv_dw(512, 512, 1),
            conv_dw(512, 512, 1),
            conv_dw(512, 512, 1),
            conv_dw(512, 512, 1),
            conv_dw(512, 512, 1),
            conv_dw(512, 1024, 2),
            conv_dw(1024, 1024, 1),
            nn.AdaptiveAvgPool2d(1)
        )
        self.fc = nn.Linear(1024, n_classes)

    def forward(self,x):
        x = self.model(x)
        x = x.view(-1,1024)
        x = self.fc(x)
        return x

In [6]:
model = MobileNet()
summary(model, input_size=(3,224,224), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 112, 112]             864
       BatchNorm2d-2         [-1, 32, 112, 112]              64
              ReLU-3         [-1, 32, 112, 112]               0
            Conv2d-4         [-1, 32, 112, 112]             288
       BatchNorm2d-5         [-1, 32, 112, 112]              64
              ReLU-6         [-1, 32, 112, 112]               0
            Conv2d-7         [-1, 64, 112, 112]           2,048
       BatchNorm2d-8         [-1, 64, 112, 112]             128
              ReLU-9         [-1, 64, 112, 112]               0
           Conv2d-10           [-1, 64, 56, 56]             576
      BatchNorm2d-11           [-1, 64, 56, 56]             128
             ReLU-12           [-1, 64, 56, 56]               0
           Conv2d-13          [-1, 128, 56, 56]           8,192
      BatchNorm2d-14          [-1, 128,

In [7]:
def train(epoch):
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        if batch_idx % 10 == 0:
            print(batch_idx, len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'%(train_loss/(batch_idx+1), 100.*correct/total, correct, total))

In [8]:
def test(epoch):
    global best_accuracy
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            ouputs = net(inputs)
            loss = criterion(ouputs, targets)

            test_loss += loss.item()
            _, predicted = ouputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            if batch_idx % 50 == 0:
                print(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'%(test_loss/(batch_idx+1), 100.*correct/total, correct, total))
        acc = 100.*correct/total
    if acc > best_accuracy:
        print('Saving..')
        best_accuracy = acc

In [ ]:
best_accuracy = 0
start_epoch = 0

net = MobileNet()
net = net.to(device)
if device == 'cuda':
  net = torch.nn.DataParallel(net)
  cudnn.benchmark = True

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.02, momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.998)

for epoch in range(start_epoch, start_epoch+251):
  train(epoch)
  test(epoch)
  scheduler.step()
  print('epoch : ',epoch, best_accuracy)

In [ ]:
for epoch in range(start_epoch, start_epoch+251):
  train(epoch)
  test(epoch)
  scheduler.step()
  print('epoch : ',epoch, best_accuracy)

In [16]:
best_accuracy

80.48

## Adam & lr-cosineAnnealingLR

In [ ]:
best_accuracy = 0
start_epoch = 0

net = MobileNet()
net = net.to(device)
if device == 'cuda':
  net = torch.nn.DataParallel(net)
  cudnn.benchmark = True

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.01)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

for epoch in range(start_epoch, start_epoch+251):
  train(epoch)
  test(epoch)
  scheduler.step()
  print('epoch : ',epoch, best_accuracy)

In [ ]:
best_accuracy

## SGD & ExponentialLR

In [ ]:
best_accuracy = 0
start_epoch = 0

net = MobileNet()
net = net.to(device)
if device == 'cuda':
  net = torch.nn.DataParallel(net)
  cudnn.benchmark = True

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.02, momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.998)

for epoch in range(start_epoch, start_epoch+251):
  train(epoch)
  test(epoch)
  scheduler.step()
  print('epoch : ',epoch, best_accuracy)

In [16]:
best_accuracy

85.23

## 배치사이즈 64

In [ ]:
best_accuracy = 0
start_epoch = 0

net = MobileNet()
net = net.to(device)
if device == 'cuda':
  net = torch.nn.DataParallel(net)
  cudnn.benchmark = True

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.02, momentum=0.9, weight_decay=5e-4)
decay_epoch = [100,150,200]
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.998)

for epoch in range(start_epoch, start_epoch+251):
  train(epoch)
  test(epoch)
  scheduler.step()
  print('epoch : ',epoch, best_accuracy)

In [21]:
best_accuracy

82.87

## momentum 0.7 weight decay 3e-4 scheduler multi

In [ ]:
best_accuracy = 0
start_epoch = 0

net = MobileNet()
net = net.to(device)
if device == 'cuda':
  net = torch.nn.DataParallel(net)
  cudnn.benchmark = True

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.02, momentum=0.7, weight_decay=3e-4)
decay_epoch = [100,150,200]
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=decay_epoch, gamma=0.1)

for epoch in range(start_epoch, start_epoch+251):
  train(epoch)
  test(epoch)
  scheduler.step()
  print('epoch : ',epoch, best_accuracy)

In [23]:
best_accuracy

81.21

## momentum 0.8 weight decay 4e-4

In [ ]:
best_accuracy = 0
start_epoch = 0

net = MobileNet()
net = net.to(device)
if device == 'cuda':
  net = torch.nn.DataParallel(net)
  cudnn.benchmark = True

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.02, momentum=0.8, weight_decay=4e-4)
decay_epoch = [100,150,200]
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=decay_epoch, gamma=0.1)

for epoch in range(start_epoch, start_epoch+251):
  train(epoch)
  test(epoch)
  scheduler.step()
  print('epoch : ',epoch, best_accuracy)

In [25]:
best_accuracy

83.33

In [ ]:
best_accuracy = 0
start_epoch = 0

net = MobileNet()
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(net.parameters(), lr=0.02, momentum=0.93, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.99)

for epoch in range(start_epoch, start_epoch+251):
    train(epoch)
    test(epoch)
    scheduler.step()
    print('epoch : ',epoch, best_accuracy)

In [ ]:
best_accuracy

In [ ]:
best_accuracy = 0
start_epoch = 0

net = MobileNet()
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(net.parameters(), lr=0.02, momentum=0.93, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.99)

for epoch in range(start_epoch, start_epoch+251):
    train(epoch)
    test(epoch)
    scheduler.step()
    print('epoch : ',epoch, best_accuracy)

In [15]:
best_accuracy

84.22

## momentum 0.95 & gamma 0.983

In [ ]:
best_accuracy = 0
start_epoch = 0

net = MobileNet()
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(net.parameters(), lr=0.02, momentum=0.95, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.983)

for epoch in range(start_epoch, start_epoch+251):
    train(epoch)
    test(epoch)
    scheduler.step()
    print('epoch : ',epoch, best_accuracy)

In [19]:
best_accuracy

84.2

In [27]:
class MobileNet(nn.Module):
    def __init__(self, ch_in=3, n_classes=10):
        super(MobileNet,self).__init__()
        momen = 0.9

        def conv_bn(inp, oup, stride):
            return nn.Sequential(
              nn.Conv2d(inp, oup, 3, stride, 1,bias=False),
              nn.BatchNorm2d(oup),
              nn.ReLU(inplace=True)
            )
        # depthwise
        def conv_dw(inp, oup, stride):
            # depthwise convolution
            # group1 : 입출력 모두 convolution
            # group2 : 2그룹으로 나누어 convolution $ concatenation
            return nn.Sequential(
              nn.Conv2d(inp, inp, 3, stride, 1, groups=inp, bias=False),
              nn.BatchNorm2d(inp),
              nn.ReLU(inplace=True),

              # pointwise
              nn.Conv2d(inp, oup, 1, 1, 0, bias=False),
              nn.BatchNorm2d(oup),
              nn.ReLU(inplace=True)
            )

        self.model = nn.Sequential(
            conv_bn(ch_in, 32, 2),
            conv_dw(32, 64, 1),
            conv_dw(64, 128, 2),
            conv_dw(128, 128, 1),
            conv_dw(128, 256, 2),
            conv_dw(256, 256, 1),
            conv_dw(256, 512, 2),
            conv_dw(512, 512, 1),
            conv_dw(512, 512, 1),
            conv_dw(512, 512, 1),
            conv_dw(512, 512, 1),
            conv_dw(512, 512, 1),
            conv_dw(512, 1024, 2),
            conv_dw(1024, 1024, 1),
            nn.AdaptiveAvgPool2d(1)
        )
        self.fc = nn.Linear(1024, n_classes)

    def forward(self,x):
        x = self.model(x)
        x = x.view(-1,1024)
        x = self.fc(x)
        return x

In [ ]:
best_accuracy = 0
start_epoch = 0

net = MobileNet()
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(net.parameters(), lr=0.02, momentum=0.9, weight_decay=5e-4)
#optimizer = optim.Adam(net.parameters(), lr=0.02, weight_decay=1e-4)

#optimizer = optim.SGD(net.parameters(), lr=0.1,momentum=0.9, weight_decay=1e-4)
#scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.983)

for epoch in range(start_epoch, start_epoch+251):
    train(epoch)
    test(epoch)
    scheduler.step()
    print('epoch : ',epoch, best_accuracy)

In [23]:
best_accuracy

82.84

In [ ]:
best_accuracy = 0
start_epoch = 0

net = MobileNet()
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.Adam(net.parameters(), lr=0.005, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.983)

for epoch in range(start_epoch, start_epoch+251):
    train(epoch)
    test(epoch)
    scheduler.step()
    print('epoch : ',epoch, best_accuracy)

In [26]:
best_accuracy

82.65

In [ ]:
best_accuracy = 0
start_epoch = 0

net = MobileNet()
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.Adam(net.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.98)

for epoch in range(start_epoch, start_epoch+251):
    train(epoch)
    test(epoch)
    scheduler.step()
    print('epoch : ',epoch, best_accuracy)

In [11]:
best_accuracy

86.37

In [ ]:
best_accuracy = 0
start_epoch = 0

net = MobileNet()
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(net.parameters(), lr=0.02, momentum=0.93, weight_decay=5e-4)
#optimizer = optim.Adam(net.parameters(), lr=0.02, weight_decay=1e-4)

#optimizer = optim.SGD(net.parameters(), lr=0.1,momentum=0.9, weight_decay=1e-4)
#scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.99)

for epoch in range(start_epoch, start_epoch+251):
    train(epoch)
    test(epoch)
    scheduler.step()
    print('epoch : ',epoch, best_accuracy)

In [13]:
best_accuracy

86.36

In [ ]:
best_accuracy = 0
start_epoch = 0

net = MobileNet()
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

criterion = nn.CrossEntropyLoss().cuda()
#optimizer = optim.SGD(net.parameters(), lr=0.02, momentum=0.9, weight_decay=5e-4)
#optimizer = optim.Adam(net.parameters(), lr=0.01, weight_decay=5e-4)
optimizer = optim.Adam(net.parameters(), lr=0.005)

#optimizer = optim.SGD(net.parameters(), lr=0.1,momentum=0.9, weight_decay=1e-4)
#scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.98)

for epoch in range(start_epoch, start_epoch+251):
    train(epoch)
    test(epoch)
    scheduler.step()
    print('epoch : ',epoch, best_accuracy)

In [15]:
best_accuracy

86.26

In [ ]:
best_accuracy = 0
start_epoch = 0

net = MobileNet()
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(net.parameters(), lr=0.02, momentum=0.9, weight_decay=5e-4)
#optimizer = optim.Adam(net.parameters(), lr=0.02, weight_decay=1e-4)

#optimizer = optim.SGD(net.parameters(), lr=0.1,momentum=0.9, weight_decay=1e-4)
#scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.99)

for epoch in range(start_epoch, start_epoch+251):
    train(epoch)
    test(epoch)
    scheduler.step()
    print('epoch : ',epoch, best_accuracy)

In [17]:
best_accuracy

86.33

In [ ]:
best_accuracy = 0
start_epoch = 0

net = MobileNet()
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

criterion = nn.CrossEntropyLoss().cuda()
#optimizer = optim.SGD(net.parameters(), lr=0.02, momentum=0.9, weight_decay=5e-4)
#optimizer = optim.Adam(net.parameters(), lr=0.01, weight_decay=5e-4)
optimizer = optim.Adam(net.parameters(), lr=0.008)

#optimizer = optim.SGD(net.parameters(), lr=0.1,momentum=0.9, weight_decay=1e-4)
#scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.98)

for epoch in range(start_epoch, start_epoch+251):
    train(epoch)
    test(epoch)
    scheduler.step()
    print('epoch : ',epoch, best_accuracy)

In [19]:
best_accuracy

86.45

In [ ]:
best_accuracy = 0
start_epoch = 0

net = MobileNet()
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.Adam(net.parameters(), lr=0.005, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.98)

for epoch in range(start_epoch, start_epoch+251):
    train(epoch)
    test(epoch)
    scheduler.step()
    print('epoch : ',epoch, best_accuracy)

In [21]:
best_accuracy

86.98

In [ ]:
best_accuracy = 0
start_epoch = 0

net = MobileNet()
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(net.parameters(), lr=0.015, momentum=0.9, weight_decay=5e-4)
#optimizer = optim.Adam(net.parameters(), lr=0.02, weight_decay=1e-4)

#optimizer = optim.SGD(net.parameters(), lr=0.1,momentum=0.9, weight_decay=1e-4)
#scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.99)

for epoch in range(start_epoch, start_epoch+251):
    train(epoch)
    test(epoch)
    scheduler.step()
    print('epoch : ',epoch, best_accuracy)

In [23]:
best_accuracy

86.0

In [ ]:
best_accuracy = 0
start_epoch = 0

net = MobileNet()
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

criterion = nn.CrossEntropyLoss().cuda()
#optimizer = optim.SGD(net.parameters(), lr=0.02, momentum=0.9, weight_decay=5e-4)
optimizer = optim.Adam(net.parameters(), lr=0.01, weight_decay=1e-4)
#optimizer = optim.Adam(net.parameters(), lr=0.008)

#optimizer = optim.SGD(net.parameters(), lr=0.1,momentum=0.9, weight_decay=1e-4)
#scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.97)

for epoch in range(start_epoch, start_epoch+251):
    train(epoch)
    test(epoch)
    scheduler.step()
    print('epoch : ',epoch, best_accuracy)

In [10]:
best_accuracy

86.05

In [ ]:
best_accuracy = 0
start_epoch = 0

net = MobileNet()
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

criterion = nn.CrossEntropyLoss().cuda()
#optimizer = optim.SGD(net.parameters(), lr=0.02, momentum=0.9, weight_decay=5e-4)
optimizer = optim.Adam(net.parameters(), lr=0.008, weight_decay=1e-5)
#optimizer = optim.Adam(net.parameters(), lr=0.008)

#optimizer = optim.SGD(net.parameters(), lr=0.1,momentum=0.9, weight_decay=1e-4)
#scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.98)

for epoch in range(start_epoch, start_epoch+251):
    train(epoch)
    test(epoch)
    scheduler.step()
    print('epoch : ',epoch, best_accuracy)

In [12]:
best_accuracy

86.67

In [ ]:
best_accuracy = 0
start_epoch = 0

net = MobileNet()
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.Adam(net.parameters(), lr=0.007, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.98)

for epoch in range(start_epoch, start_epoch+251):
    train(epoch)
    test(epoch)
    scheduler.step()
    print('epoch : ',epoch, best_accuracy)

In [16]:
best_accuracy

87.07